# 基本设置

In [1]:
import os
import pandas as pd

import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
# import pre_cor_1
# import pre_cor
from toolkits.nlp import pre_cor_circ
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.205 seconds.
Prefix dict has been built succesfully.


In [2]:
def get_txt_encode(file_path):
    '''
    Function:
        获取txt文档的编码格式
    Arguments:
        file_path -> string -- 文档所在路径
    Return:
        encode -> string -- txt文档的编码格式
    '''
    decode_set = ['utf-8','gb18030','ISO-8859-2','gb2312','gbk', 'ANSI'] # 编码集
    encode = 0
    for k in decode_set:
        try :
            file = open(file_path, 'r', encoding = k).read()
            # print('\n\t读取成功： %s \n\t编码方式为： %s' %(file_path, k))
            encode = k
            break
        except :
            continue
    if encode:
        return encode
    else :
        return 'No encode'

In [24]:
def combined_data(data_folder, save_filename, label = None):
    file_list = os.listdir(data_folder)
    combined_data = []
    for file in file_list:
        tmp = os.getcwd() + '\\' + data_folder + '\\' + file
        encoding = get_txt_encode(tmp)
        f = open(tmp, "r+", encoding = encoding)
        title = file.replace('.txt', '').split('_')[1]
        content = ''
        for line in f:
            content += line.strip()
        f.close()
        combined_data.append([title,content])
    data = pd.DataFrame(combined_data, columns = ['title', 'content'])
    data['label'] = label
    data.to_excel(save_filename, index=False)
    
circ_neg_folder = 'data/20181112/0604-0608保监会非负补充'
circ_neg_filename = 'data/20181112/0604-0608保监会非负补充.xlsx'
combined_data(circ_neg_folder, circ_neg_filename, label = 0)
# circ_pos_folder = 'data/20180619/0604-0608保监会误判为负面的数据'
# circ_pos_filename = 'data/circ_0905_pos_raw_data.xlsx'
# combined_data(circ_pos_folder, circ_pos_filename, label = 1)
# circ_pos_folder = 'data/20180619/0604-0608保监会非负补充'
# circ_pos_filename = 'data/circ_0905_pos_raw_data_add.xlsx'
# combined_data(circ_pos_folder, circ_pos_filename, label = 1)

# 导入数据

In [4]:
forder = '20181112'

## 负面数据

In [45]:
file_list_1 = ['0101-0228保监会负面_人工校正.xlsx',
               '0301-0521保监会负面_人工校正.xlsx',
               '0502-0507保监会负面.xlsx',
               '0604-0608保监会负面.xlsx']

In [46]:
cir_data_raw_1 = pd.DataFrame()
for file_name in file_list_1:
    currentPath  = 'data/{0}/{1}'.format(forder, file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
#         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
        print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#         tmp_data = tmp_data[tmp_data['备注'] != '删除']
#         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
        print('去空值后： ', tmp_data.shape, file_name, sheet_name)
        cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

cir_data_raw_1['label'] = -1 # cir_data_raw_1['label'].apply(lambda x:x.strip())    
# cir_data_raw_1 = cir_data_raw_1[['label', 'title', 'content']]
# cir_data_raw_1.columns = ['label', 'title', 'content']
print('总数：', cir_data_raw_1.shape)
cir_data_raw_1.head()

去空值前：  (905, 2) 0101-0228保监会负面_人工校正.xlsx 1-2月份数据
去空值后：  (905, 2) 0101-0228保监会负面_人工校正.xlsx 1-2月份数据
去空值前：  (1110, 2) 0301-0521保监会负面_人工校正.xlsx 3月1号到5月21号
去空值后：  (1110, 2) 0301-0521保监会负面_人工校正.xlsx 3月1号到5月21号
去空值前：  (174, 3) 0502-0507保监会负面.xlsx Sheet1
去空值后：  (174, 3) 0502-0507保监会负面.xlsx Sheet1
去空值前：  (321, 3) 0604-0608保监会负面.xlsx Sheet1
去空值后：  (321, 3) 0604-0608保监会负面.xlsx Sheet1
总数： (2510, 3)


,content,label,title
0,"\n\t\t\t\t\t <font color=""#FF0000"">保监会</font...",-1,根据决定书，国寿财险瑞金支公司存在以下违法行为：
1,"\n\t\t\t\t\t 随着<font color=""#FF0000"">寿险</fon...",-1,转型阵痛 净利下滑
2,\t\t#监督# 【中华联合产险江西现两宗违法 给投保人约定以外利益】保监会网站公布江西保监...,-1,\t\t#监督# 【中华联合产险江西现两宗违法 给投保人约定以外利益】保监会网站公布江西保监...
3,\t\t【4季度偿付能力报告：珠江人寿风险综合评级再降级】截至2月6日，百余家保险公司已经悉...,-1,\t\t【4季度偿付能力报告：珠江人寿风险综合评级再降级】截至2月6日，百余家保险公司已经悉...
4,【偿付能力再降 中法<em class=“red“>人寿</em>深陷困局】业内常年垫底的中...,-1,\t\t【偿付能力再降 中法<em class=“red“>人寿</em>深陷困局】业内常年...


In [47]:
cir_data_raw_1['label'].value_counts()

-1    2510
Name: label, dtype: int64

## 正面数据

In [48]:
file_list_2 = ['0604-0608保监会非负补充.xlsx',
               '0604-0608保监会误判为负面的数据.xlsx',
               '180518保监会非负-来自校正非负-提交许欢.xlsx',]

In [49]:
cir_data_raw_2 = pd.DataFrame()
for file_name in file_list_2:
    currentPath  = 'data/{0}/{1}'.format(forder, file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
#         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
        print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#         tmp_data = tmp_data[tmp_data['备注'] != '删除']
#         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
        print('去空值后： ', tmp_data.shape, file_name, sheet_name)
        cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)

cir_data_raw_2['label'] = 0 # cir_data_raw_1['label'].apply(lambda x:x.strip())    
# cir_data_raw_2 = cir_data_raw_2[['label', 'title', 'content']]
# cir_data_raw_1.columns = ['label', 'title', 'content']
print('总数：', cir_data_raw_2.shape)
cir_data_raw_2.head()

去空值前：  (183, 3) 0604-0608保监会非负补充.xlsx Sheet1
去空值后：  (183, 3) 0604-0608保监会非负补充.xlsx Sheet1
去空值前：  (40, 3) 0604-0608保监会误判为负面的数据.xlsx Sheet1
去空值后：  (40, 3) 0604-0608保监会误判为负面的数据.xlsx Sheet1
去空值前：  (883, 3) 180518保监会非负-来自校正非负-提交许欢.xlsx Sheet1
去空值后：  (883, 3) 180518保监会非负-来自校正非负-提交许欢.xlsx Sheet1
总数： (1106, 3)


,title,content,label
0,数数中国平安的硬家底,数数中国平安的硬家底中国第一强寿险 平安寿险 中国第二大财险 平安财险 中国三流冲二流的银行...,0
1,分公司举办反腐倡廉教育专题讲座,分公司举办反腐倡廉教育专题讲座为了进一步落实总公司反欺诈工作会议精神，加强反内部舞弊工作和员...,0
2,医生告诉你人生第一份保险该怎么买,医生告诉你：人生第一份保险该怎么买？刘先生，33周岁，无社保，于2016年8月24日投保了医...,0
3,中小企业网税延养老险分层满足多元需,中小企业网:税延养老险“分层”满足多元需求个人税收递延型商业养老保险（以下简称“税延养老保险...,0
4,快讯受惠政策免税内险股大涨中国太平一,快讯：受惠政策免税内险股大涨 中国太平一度涨超7%确定不再关注此人吗新浪港股讯 保险公司一季...,0


In [50]:
cir_data_raw_2['label'].value_counts()

0    1106
Name: label, dtype: int64

In [51]:
fea_filename = 'data/circ_tendency_data.xlsx'
cir_data_raw = pd.concat([cir_data_raw_1, cir_data_raw_2], axis = 0)
cir_data_raw['title_content'] = cir_data_raw['title'].astype(str) + '。' + cir_data_raw['content'].astype(str)
print(cir_data_raw['label'].value_counts())
with pd.ExcelWriter(fea_filename) as writer:
    cir_data_raw.to_excel(writer,'tendency', index = False)
    writer.save()

-1    2510
 0    1106
Name: label, dtype: int64


## 合并数据

In [52]:
cir_data_raw = pd.concat([cir_data_raw_1, cir_data_raw_2],
                          #cir_data_raw_3], 
                         axis = 0)

# cir_data_raw = cir_data_raw_2
cir_data_raw['title_content'] = cir_data_raw['title'].astype(str) + '。' + cir_data_raw['content'].astype(str)
# cir_data_raw['label'] = cir_data_raw['label'].apply(lambda x:x.strip())
cir_data_raw['index'] = range(cir_data_raw.shape[0])
print('去重前： ', cir_data_raw.shape)
cir_data_raw = cir_data_raw.drop_duplicates(subset = 'title_content')
print('去重后： ', cir_data_raw.shape)
cir_data_raw.head()

去重前：  (3616, 5)
去重后：  (3608, 5)


,content,label,title,title_content,index
0,"\n\t\t\t\t\t <font color=""#FF0000"">保监会</font...",-1,根据决定书，国寿财险瑞金支公司存在以下违法行为：,根据决定书，国寿财险瑞金支公司存在以下违法行为：。 \n\t\t\t\t\t <fon...,0
1,"\n\t\t\t\t\t 随着<font color=""#FF0000"">寿险</fon...",-1,转型阵痛 净利下滑,"转型阵痛 净利下滑。 \n\t\t\t\t\t 随着<font color=""#FF0...",1
2,\t\t#监督# 【中华联合产险江西现两宗违法 给投保人约定以外利益】保监会网站公布江西保监...,-1,\t\t#监督# 【中华联合产险江西现两宗违法 给投保人约定以外利益】保监会网站公布江西保监...,\t\t#监督# 【中华联合产险江西现两宗违法 给投保人约定以外利益】保监会网站公布江西保监...,2
3,\t\t【4季度偿付能力报告：珠江人寿风险综合评级再降级】截至2月6日，百余家保险公司已经悉...,-1,\t\t【4季度偿付能力报告：珠江人寿风险综合评级再降级】截至2月6日，百余家保险公司已经悉...,\t\t【4季度偿付能力报告：珠江人寿风险综合评级再降级】截至2月6日，百余家保险公司已经悉...,3
4,【偿付能力再降 中法<em class=“red“>人寿</em>深陷困局】业内常年垫底的中...,-1,\t\t【偿付能力再降 中法<em class=“red“>人寿</em>深陷困局】业内常年...,\t\t【偿付能力再降 中法<em class=“red“>人寿</em>深陷困局】业内常年...,4


In [53]:
print(cir_data_raw.shape)
cir_data_raw.groupby('label')['title'].count()

(3608, 5)


label
-1    2502
 0    1103
Name: title, dtype: int64

In [54]:
import re

In [58]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [60]:
s = 'sss<font color="#FF0000">kkk' 
print(s)
print(re.sub('<font color="#FF0000">', '', s))

sss<font color="#FF0000">kkk
ssskkk


# 预处理

In [49]:
titles = pre_cor_circ.handle_contents(cir_data_raw['title'].tolist())
print(len(titles))

save_filename = 'data/titles.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in titles:
    fid.write(data + '\n')
fid.close()

623


In [50]:
contents = pre_cor_circ.handle_contents(cir_data_raw['content'].tolist())
print(len(contents))
print(contents[:2])
# contents = [re.sub(r'[a-z]*', '', x) for x in contents]
# print(len(contents))
# print(contents[:2])

coprus_save_filename = 'data/contents.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in contents:
    f.write(data + '\n')
f.close()

623
['股份 转 股东 不知情 原 保监会 审查 批复 遭 质疑 股东 私自 转让 公司 股权 违法 工商部门 吊销 营业执照 股东 申请 法院 强制 清算 发现 公司 早 年前 股份 转让 公司 并未 收到 转让款 申请 政府 信息 公开 部门 并未 法定期限 答复 股东 行政诉讼 邮件 提供 少部分 信息 涉及 股东 利益 大部分 信息 并未 提供 大连人 李岩 接受 獬豸 新闻 ID 记者 采访 时 谈起 此事 摇头 李岩 说 做梦 想到 身为 北京 毕保信 投资 有限公司 下称 毕保信 公司 股东 未有 知情 情况 持 股份 公司 非法 转让 毕保信 公司 未 收到 分文 股权 转让款 毕保信 公司 成立 年 月 日 工商部门 认缴 出资额 万元 股东 林建伟 出资 万元 持有 毕保信 公司 股权 股东 李岩 出资 万元 持有 股权 担任 公司 监事 期间 毕保信 公司 经营 年 月初 李岩 发现 毕保信 公司 营业执照 工商部门 予以 强制 吊销 同年 月 日 李岩 毕保信 公司 吊销 营业执照 超过 法定期限 未 成立 清算组 清算 北京市 第三 中级 人民法院 下称 北京 三中 院 申请 毕保信 公司 强制 清算 法院 审理 查明 毕保信 公司 早 年 月 日 北京市工商局 以未 期限内 接受 年度 企业 年检 吊销 营业执照 责令 股东 清算组 清算 债权债务 办理 注销 登记 年 月 日 北京 三中 院 民事裁定 毕保信 公司 成立 强制 清算组 清算 过程 中 李岩 得知 早 年 月 日 毕保信 公司 安达 北美洲 FF 控股公司 下称 安达 北美洲 FF 华泰财产保险 股份 有限公司 下称 华泰 公司 签订 股份 转让 协议 协议 约定 安达 北美洲 受让 毕保信 公司 持有 华泰 公司 股份 每股 价格 美元 共计 美元 合计 人民币 元 对此 李岩 称 并未 股权 转让 事宜 做出 参加 股东会 决议 毕保信 公司 未 收到 股权 转让款 毕保信 公司 股东 股权 转让 致使 毕保信 公司 股份 对价 资产 丧失 侵害到 股东 毕保信 公司 享有 盈余 分配权 剩余 财产 索取权 股权 权能 李岩 北京 三中 院 提起 侵权 诉讼 李岩 说 毕保信 公司 持股 FF 华泰财产保险 股份 有限公司 简称 华泰 公司 原始股 原始股 股东 

In [52]:
label = cir_data_raw['label'].tolist()
print(len(label))

coprus_save_filename = 'data/labels.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in label:
    f.write(str(data) + '\n')
f.close()

623


In [54]:
cir_data_raw.to_excel('data/result/title_content_label.xlsx', index = False)
cir_data_raw.shape

(623, 5)